### Linear Regression Model to Predict Future Year Prices (selecting from all features)
#### Read in all merged csv data files

In [16]:
import pandas as pd
import glob
import os

path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016'], axis=1, inplace=True)
merged_df

,year,sa2_2021,residence_type,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,min_distance_to_poli,min_distance_to_shop,weekly_rent,gdp(USD Millioins),saving_rate(% of GDP),income_per_person,population_density,crime_cases
0,2013,204011057,House,2.0,1.0,0,227.97163,23.16035,7.35747,16.96507,35.56825,21.35025,22.04660,9.35209,300.0,1536454,6.861393,39683.563449,2.172408,86.0
1,2013,205051101,House,2.0,1.0,0,223.66084,5.71742,6.50536,6.76794,7.54355,7.42972,6.28177,9.35209,215.0,1536454,6.861393,47222.702327,5.425503,36.0
2,2013,204011057,House,2.0,1.0,0,243.25680,5.11222,0.20027,36.72106,50.85341,36.63541,0.08478,9.35209,175.0,1536454,6.861393,39683.563449,2.172408,86.0
3,2013,202011022,House,4.0,2.0,0,140.35827,78.32509,10.66523,11.91899,11.26906,177.44731,84.47341,9.35209,350.0,1536454,6.861393,43556.283562,473.765281,1288.0
4,2013,208041195,Apartment,1.0,1.0,0,13.86135,0.93250,1.32931,3.49174,2.20800,177.44731,84.47341,3.96501,275.0,1536454,6.861393,86103.411528,2834.210526,1923.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172030,2022,205021086,House,3.0,1.0,1,293.28053,0.56012,1.21809,114.77016,90.08591,140.56888,74.35608,13.64920,265.0,3305754,12.839000,54365.266130,402.000000,281.0
172031,2022,217041479,House,3.0,2.0,2,258.29111,3.49087,5.08707,3.60570,8.37185,2.60312,74.35608,13.64920,500.0,3305754,12.839000,60828.473189,689.000000,3049.0
172032,2022,208021177,House,2.0,2.0,1,9.47077,2.45011,1.33931,1.62322,3.63291,140.56888,74.35608,1.97636,750.0,3305754,12.839000,98756.492866,3656.000000,759.0
172033,2022,206041506,Apartment,1.0,1.0,1,1.84933,0.65199,1.10438,1.27940,1.87840,140.56888,74.35608,13.64920,409.0,3305754,12.839000,71305.473808,5791.000000,1788.0


In [25]:
merged_df.rename({'gdp(USD Millioins)': 'gdp', 'saving_rate(% of GDP)': 'saving_rate'}, axis=1, inplace=True)
ALL_FEATURES = set(merged_df.drop(['weekly_rent', 'year', 'residence_type', 'sa2_2021'], axis=1).columns)
ALL_FEATURES

{'crime_cases',
 'gdp',
 'income_per_person',
 'min_distance_to_cbd',
 'min_distance_to_hosp',
 'min_distance_to_park',
 'min_distance_to_poli',
 'min_distance_to_prim',
 'min_distance_to_second',
 'min_distance_to_shop',
 'min_distance_to_train',
 'nbath',
 'nbed',
 'ncar',
 'population_density',
 'saving_rate'}

In [2]:
#import seaborn as sns
#sns.pairplot(merged_df)

### Log transformation provides better performance

In [3]:
import numpy as np

LOG_FEATURES = ['saving_rate', 'min_distance_to_prim', 
'min_distance_to_poli', 'min_distance_to_park', 'min_distance_to_second', 'min_distance_to_hosp', 'min_distance_to_cbd', 
'min_distance_to_shop', 'population_density', 'income_per_person', 
'crime_cases', 'min_distance_to_train', 'gdp']
for log_feature in LOG_FEATURES:
    merged_df[log_feature] = np.log(merged_df[log_feature])

merged_df['weekly_rent'] = np.log(merged_df['weekly_rent'])

In [4]:
print(merged_df.columns)
#pd.get_dummies(merged_df['sa2_2021'])
merged_df = pd.get_dummies(data=merged_df, columns=['sa2_2021'], prefix='sa2')
merged_df = pd.get_dummies(data=merged_df, columns=['residence_type'], prefix='resiType') 
#merged_df.drop(['sa2_2021', 'residence_type'], axis=1, inplace=True)
merged_df.dropna(inplace=True)

y = merged_df['weekly_rent']
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(merged_df.isna().sum())
X = merged_df.drop(['weekly_rent'], axis=1)
X

Index(['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar',
       'min_distance_to_cbd', 'min_distance_to_park', 'min_distance_to_prim',
       'min_distance_to_second', 'min_distance_to_train',
       'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop',
       'weekly_rent', 'gdp', 'saving_rate', 'income_per_person',
       'population_density', 'crime_cases'],
      dtype='object')


,year,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,...,sa2_217031473,sa2_217031474,sa2_217031475,sa2_217031476,sa2_217041477,sa2_217041478,sa2_217041479,sa2_217041480,resiType_Apartment,resiType_House
0,2013,2.0,1.0,0,5.429221,3.142442,1.995716,2.831157,3.571453,3.061063,...,0,0,0,0,0,0,0,0,0,1
1,2013,2.0,1.0,0,5.410131,1.743518,1.872626,1.912197,2.020693,2.005488,...,0,0,0,0,0,0,0,0,0,1
2,2013,2.0,1.0,0,5.494118,1.631634,-1.608089,3.603350,3.928947,3.601015,...,0,0,0,0,0,0,0,0,0,1
3,2013,4.0,2.0,0,4.944198,4.360868,2.366989,2.478133,2.422061,5.178674,...,0,0,0,0,0,0,0,0,0,1
4,2013,1.0,1.0,0,2.629104,-0.069886,0.284660,1.250400,0.792087,5.178674,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172030,2022,3.0,1.0,1,5.681130,-0.579604,0.197284,4.742932,4.500764,4.945698,...,0,0,0,0,0,0,0,0,0,1
172031,2022,3.0,2.0,2,5.554087,1.250151,1.626702,1.282516,2.124875,0.956711,...,0,0,0,0,0,0,1,0,0,1
172032,2022,2.0,2.0,1,2.248210,0.896133,0.292155,0.484412,1.290034,4.945698,...,0,0,0,0,0,0,0,0,0,1
172033,2022,1.0,1.0,1,0.614823,-0.427726,0.099284,0.246391,0.630420,4.945698,...,0,0,0,0,0,0,0,0,1,0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_train

,year,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,...,sa2_217031473,sa2_217031474,sa2_217031475,sa2_217031476,sa2_217041477,sa2_217041478,sa2_217041479,sa2_217041480,resiType_Apartment,resiType_House
124865,2022,3.0,1.0,1,5.031447,2.899481,-1.247716,-0.542936,0.133569,4.945698,...,0,0,0,0,0,0,0,0,0,1
104159,2021,4.0,2.0,4,5.046102,1.404505,0.859356,1.009347,4.498420,4.955256,...,0,0,0,0,0,0,0,0,0,1
11046,2014,4.0,1.0,3,2.890546,0.069731,0.343100,5.171704,0.254851,5.178674,...,0,0,0,0,0,0,0,0,0,1
67719,2019,2.0,1.0,1,2.335963,0.609744,-0.065104,0.252065,0.385371,-0.530994,...,0,0,0,0,0,0,0,0,0,1
25343,2015,4.0,2.0,2,4.576933,-0.621199,1.368695,3.024279,2.620736,5.178674,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119894,2022,2.0,1.0,1,2.625978,-1.147403,0.092953,4.742932,0.767679,4.945698,...,0,0,0,0,0,0,0,0,1,0
103709,2021,2.0,1.0,1,5.299019,-0.265621,-1.791080,2.978110,-0.736535,4.955256,...,0,0,0,0,0,0,0,0,0,1
131947,2022,3.0,2.0,2,3.587618,-0.562996,0.619802,4.742932,4.500764,4.945698,...,0,0,0,0,0,0,0,0,0,1
146884,2022,3.0,1.0,1,4.129493,-2.575313,-1.160466,1.256692,1.113576,4.945698,...,0,0,0,0,0,0,0,0,0,1


#### Prepare a null model that uses only Year, SA2 code, Residence Type to predict weekly rent

In [6]:
#external_X = merged_df.drop(list(merged_df.filter(regex='resiType')), axis=1)
#external_X.drop(list(external_X.filter(regex='sa2')), axis=1, inplace=True)
#external_X.drop(['nbed', 'nbath', 'ncar', 'weekly_rent'], axis=1, inplace=True)
all_candidates = ['nbed', 'nbath', 'ncar', 'min_distance_to_cbd', 'min_distance_to_park',
       'min_distance_to_prim', 'min_distance_to_second',
       'min_distance_to_train', 'min_distance_to_hosp', 'min_distance_to_poli',
       'min_distance_to_shop', 'gdp', 'saving_rate', 'income_per_person',
       'population_density', 'crime_cases']
null_X = X_train[['year']+list(X_train.filter(regex='sa2'))+list(X_train.filter(regex='resiType'))] # null predictors
null_X

,year,sa2_201011001,sa2_201011002,sa2_201011005,sa2_201011006,sa2_201011007,sa2_201011008,sa2_201011481,sa2_201011482,sa2_201011483,...,sa2_217031473,sa2_217031474,sa2_217031475,sa2_217031476,sa2_217041477,sa2_217041478,sa2_217041479,sa2_217041480,resiType_Apartment,resiType_House
124865,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
104159,2021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11046,2014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
67719,2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25343,2015,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119894,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
103709,2021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
131947,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
146884,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

null_model = sm.OLS(y_train, null_X).fit()
# Summary of all factors
null_summary = null_model.summary()

In [8]:
print(null_summary)
print(f'AIC = {null_model.aic}')

                            OLS Regression Results                            
Dep. Variable:            weekly_rent   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     217.2
Date:                Tue, 04 Oct 2022   Prob (F-statistic):               0.00
Time:                        23:03:57   Log-Likelihood:                -15936.
No. Observations:              120412   AIC:                         3.288e+04
Df Residuals:                  119906   BIC:                         3.779e+04
Df Model:                         505                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
year                   0.0339      0

#### (All Features) Forward Selection - to add the most useful predictor that gives the lowest AIC at each iteration

In [9]:
AIC_dict = {}
last_min = null_model.aic
candidates = []

while(True):
    for x in all_candidates:
        print(f"trying feature {x}")
        new_X = X_train[x]
        forward_X = pd.concat([new_X, null_X], axis=1)
        model = sm.OLS(y_train, forward_X).fit()
        AIC_dict[x] = model.aic
        print(f"AIC = {model.aic}")

    min_aic =  min(AIC_dict.values())
    min_aic_key = min(AIC_dict, key=AIC_dict.get)

    if min_aic < last_min:
        candidates.append(min_aic_key)
        all_candidates.remove(min_aic_key)
        last_min = min_aic
        null_X = pd.concat([X_train[min_aic_key], null_X], axis=1)

        print('step: ' + str(len(candidates)))
        print(candidates)
        print('new AIC: ' + str(min_aic))
        print('===============')
    else:
        model = sm.OLS(y_train, null_X).fit()
        print(model.summary())
        break

trying feature nbed
AIC = 5537.476595479879
trying feature nbath
AIC = 14423.58554619539
trying feature ncar
AIC = 28320.23791462253
trying feature min_distance_to_cbd
AIC = 32884.43074500706
trying feature min_distance_to_park
AIC = 32841.16356234002
trying feature min_distance_to_prim
AIC = 32458.700972849736
trying feature min_distance_to_second
AIC = 32743.00104618637
trying feature min_distance_to_train
AIC = 32705.395217711222
trying feature min_distance_to_hosp
AIC = 32824.27053061465
trying feature min_distance_to_poli
AIC = 32794.937256567704
trying feature min_distance_to_shop
AIC = 32884.81204473466
trying feature gdp
AIC = 32791.9835198822
trying feature saving_rate
AIC = 32784.77242556872
trying feature income_per_person
AIC = 32603.501739227155
trying feature population_density
AIC = 32187.179800447368
trying feature crime_cases
AIC = 32878.79913433769
step: 1
['nbed']
new AIC: 5537.476595479879
trying feature nbath
AIC = -1946.7652280156617
trying feature ncar
AIC = 4546

#### If standardise is required, check this out https://www.youtube.com/watch?v=QH_elD_JKuc&list=PLe9UEU4oeAuV7RtCbL76hca5ELO_IELk4&index=10

In [23]:
SELECTED_FEATURES = set(candidates)
sorted(SELECTED_FEATURES)

['crime_cases',
 'income_per_person',
 'min_distance_to_cbd',
 'min_distance_to_park',
 'min_distance_to_poli',
 'min_distance_to_prim',
 'min_distance_to_second',
 'min_distance_to_shop',
 'min_distance_to_train',
 'nbath',
 'nbed',
 'ncar',
 'population_density',
 'saving_rate']

In [26]:
import numpy as np
drop = list(ALL_FEATURES - SELECTED_FEATURES)
predictions = model.predict(X_test.drop(drop, axis=1))
errors = np.array(predictions - y_test)
squared_errors = errors**2
mean_squared_error = squared_errors.mean()

print(f'MSE: {mean_squared_error}')

tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

MSE: 0.05961888735421223
Model R^2: 0.5891


In [12]:
#lm = sm.OLS(y, X.drop(['min_distance_to_park', 'min_distance_to_second'], axis=1)).fit()

In [13]:
#import pickle
#pickle.dump(lm, open('../web/models/lr_rental_model.pkl','wb'))
#model = pickle.load(open('../web/models/lr_rental_model.pkl','rb'))